### WRF CHEM compiling in ARM architecture, Beagle Bone Black

This is based on http://www2.mmm.ucar.edu/wrf/OnLineTutorial/index.php
#### compiler test
went successful, except in test4 using 64 bit
### libraries
#### Netcdf compiling
1. Environment has to be properly setup, following was correct for ARM arch, clarified based on [this](http://www.unidata.ucar.edu/software/netcdf/docs/netcdf-install/Environment.html#Environment)
```fortran
export DIR=/media/card/wrf/lib
export CC=gcc
export CXX=g++
export FC=gfortran
export FCFLAGS=-g
export F77=gfortran
export FFLAGS=-g
export CXXFLAGS=-g
```
2. Faced error in ```make``` after configure command of ```./configure --prefix=$DIR/netcdf --disable-dap \
     --disable-netcdf-4 --disable-shared```. The errors are like ```./libtool: line 1133: g++: command not found```, ```libtool: link: unsupported hardcode properties```( solved by sudo apt-get install g++-mingw-w64-i686), with blocking in folder CXX ```ar: libnetcdf_c___la-netcdf.o: No such file or directory``` (solved by specifying correct environment variables as above). Following packages were also forced to install by errors and for searches while making.
```fortran
sudo apt-get install g++-mingw-w64-i686
sudo apt-get install build-essential
sudo apt-get install csh
sudo apt-get install gfortran
```
3. Finally after making correct environment, the command ```make``` finished without error and so the ```make install```, a folder named netcdf was created and executables are in the folders. As per the [page](http://www.unidata.ucar.edu/software/netcdf/docs/netcdf-install/Environment.html#Environment), environments were again made for ```export PATH=$DIR/netcdf/bin:$PATH``` and ```export NETCDF=$DIR/netcdf```. After this environments set up, the netcdf compiling test was made using ```nc-config --version``` and ```nc-config --prefix```, it given the correct answers.
4. After reboot of BBB, all the environment setup is removed. it was returning blank line for ```echo $DIR```. For instance, the libraries are compiled in sd card of the BBB. The sd card is mounted to BBB using the command ```sudo mount -t ext4 /dev/mmcblk0p2 /media/card``` based on [this](http://unix.stackexchange.com/questions/118090/mounting-mmcblk0p1-failed-with-invalid-argument). Then again all the environment set up is executed only nc-config worked properly, notably command like ```export DIR=/media/card/wrf/lib, export PATH=$DIR/netcdf/bin:$PATH, export NETCDF=$DIR/netcdf```

#### compiling mpich

1. Even if the ssh made with BBB is broken, the environment set up was removed, it was pretend to re apply the export command as specified above. MPICH for cluster parallel execution was compiled as per the [page](http://www2.mmm.ucar.edu/wrf/OnLineTutorial/index.php) .
2. There were a error saying ```mpich type_get_attr.lo is not a valid libtool object```, while ```make``` after configure by executing the command ```./configure --prefix=$DIR/mpich```. 
3. Based on [this](http://stackoverflow.com/questions/21642600/libtool-did-not-generate-a-lo-file-when-everything-is-there) latest libtool in Debian package was ```dpkg -i``` installed by removing the older version available with BBB.
4. After this, ```make``` ended without any error and as well the ```make install```. A environment is setup for mpich by ```export PATH=$DIR/mpich/bin:$PATH```. 

#### libraries for WPS: zlib,libpng,JasPer

1. These libraries were installed without any error as per the [page](http://www2.mmm.ucar.edu/wrf/OnLineTutorial/index.php)

#### Library Compatibility Tests
1. compatibility test such as Fortran + C + NetCDF and Fortran + C + NetCDF + MPI was carried out and it is executed without any error. 

#### compiling WRFV3.6
1. As per the [page](http://www2.mmm.ucar.edu/wrf/OnLineTutorial/index.php), the source folder of WRFV3 was unzipped and cd into it and run ```./configure``` but ends with blank list of platform to chosen, that way the conifugre script has to be ends with abruptly.
2. Based on [this](http://supersmith.com/site/ARM_files/wrf_on_arm.pdf) work of porting WRF for ARM, it is found that file ```configure_new.defaults``` in arch folder of source has to be edited to replace word x86_64 with armv7l for gfortran compiler section. Note the word x86_64 has to be replaced only along with gfortran, other wise it will be problematic.
3. With this edit the problem of blank platform list was solved and now the ```./configure```ends with warning saying that ```W A R N I N G There are some Fortran 2003 features in WRF that your compiler does not recognize. The IEEE signaling call has been removed.  That may not be enough```. This was referred in this forum [post](http://forum.wrfforum.com/viewtopic.php?f=5&t=5703) indicating a minimal error due to compiler version. The current version of gfortran is 4.7 (from [here](https://packages.debian.org/wheezy/armhf/gfortran-4.7/download))and BBB is having 4.6.3 version. 
4. New version deb file of gfortran is downloaded for armhf and dpkg on this deb gives rise to a error saying that gcc also older version of 4.6.3. But faced problem in updating the gcc and gfortran, update was end with error saying update was expired. The error was resolved using this command based on this [page](http://unix.stackexchange.com/questions/2544/how-to-work-around-apts-release-file-expired-problem-on-a-local-mirror). To list installed pages in debian used ```dpkg-query -l```. After a failed attempt for installing gfortran 4.7 and gcc, it is felt to stick to the wrf compile by neglecting the latter warning as said in this [page](http://forum.wrfforum.com/viewtopic.php?f=5&t=5703)
5. wrf compile restarted after rectifying the earlier error of no installation of libcloog-ppl while compilation of WRF. In current attempt it was followed the notes of this [page](http://sergioibarra.blogspot.in/2012/10/wrf-chem-compilacion-en-ubuntu-1204.html) for compiling WRF CHEM by adding environment in /etc/bash.bashrc file. This page is reference of installing libcloog-ppl is used for solving the error.
6. This attempt of compiling wrf also has the problem of errors such as ```Fatal Error: Can't open module file 'module_configure.mod' for reading at (1): No such file or directory```, this error is continuousness for multiple module. This typical error is referred in this [page](http://forum.wrfforum.com/viewtopic.php?f=5&t=897) indicating due to mismatch of compiler versions. An attempt is made with WRFV 3.4.1 and chem code as done for HP laptop also ending with this similar error. The attempt to compile WRF CHEM in BBB is abruptly ending here due to time constraints.

#### Conclusion

It can be concluded that WRF-CHEM most possibly would compile in ARM cores as noted [here](www.supersmith.com/site/ARM_files/wrf_on_arm.pdf). The compilers version (gcc, gfrotran, cpp), version of WRF and Debian distribution will be the major bottle neck for that.

Based on 
1. http://forums.debian.net/viewtopic.php?f=10&t=2425
2. http://ubuntuforums.org/showthread.php?t=1422476
3. https://github.com/saconswl/Real_time_air_pollution_Mod_Proj-2013-2014/blob/home/working_notes/wn_2014-05/Installing_netcdf_python_in_Ubuntu12.04.md
4. http://ubuntuforums.org/showthread.php?t=1464768
5. http://www.unidata.ucar.edu/support/help/MailArchives/netcdf/msg10293.html
